In [1]:
import cv2
import os
import numpy as np
import random

In [2]:
input_root = "dataset"
output_root = "augmented_dataset"

In [5]:
# Computing the minimum number of clean images per class
minCleanImages = 9999999999
for class_name in os.listdir(input_root):
    class_path = os.path.join(input_root, class_name)
    if not os.path.isdir(class_path):
        continue

    # Create class folder inside augmented_dataset
    output_class_path = os.path.join(output_root, class_name)
    os.makedirs(output_class_path, exist_ok=True)
    counter = 0
    # Checking the number of images that are clean
    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)
        img = cv2.imread(img_path)
        if img is None:
            continue
        counter+=1
    minCleanImages = min(minCleanImages, counter)    
    print(f"Processed {counter} images in class '{class_name}'")
print(f"Minimum number of clean images across classes: {minCleanImages}")

Processed 247 images in class 'cardboard'
Processed 385 images in class 'glass'
Processed 315 images in class 'metal'
Processed 449 images in class 'paper'
Processed 363 images in class 'plastic'
Processed 106 images in class 'trash'
Minimum number of clean images across classes: 106


When augmenting the data, we want the sample size for each class to be similar, so we decided to set a limit for the augmentation size.

In [ ]:
#Augmentation Techniques
def rotate_image(img, angle):
    h, w = img.shape[:2]
    M = cv2.getRotationMatrix2D((w/2, h/2), angle, 1)
    return cv2.warpAffine(img, M, (w, h))

def add_gaussian_noise(img):
    mean = 0
    std = 25      # adjust noise level if needed
    noise = np.random.normal(mean, std, img.shape).astype(np.uint8)
    noisy_img = cv2.add(img, noise)
    return noisy_img

def change_brightness(img):
    # Random brightness adjustment between -50 and +50
    value = random.randint(-50, 50)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    # Clip brightness values to stay valid (0–255)
    v = np.clip(v + value, 0, 255)

    final_hsv = cv2.merge((h, s, v))
    bright_img = cv2.cvtColor(final_hsv, cv2.COLOR_HSV2BGR)
    return bright_img

In [ ]:
numberOfAugmentationTecniques = 4 

# Loop over classes
for class_name in os.listdir(input_root):
    class_path = os.path.join(input_root, class_name)
    if not os.path.isdir(class_path):
        continue

    # Create class folder inside augmented_dataset
    output_class_path = os.path.join(output_root, class_name)
    os.makedirs(output_class_path, exist_ok=True)
    # duplicating original images and updating their name
    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)
        img = cv2.imread(img_path)

        if img is None:
            continue
        filename = os.path.splitext(img_name)[0]
        # Save original image
        cv2.imwrite(os.path.join(output_class_path, f"{filename}_orig.png"), img)
    # Loop over images
    cnt = len(os.listdir(class_path))
    for img_name in os.listdir(class_path):
        img_path = os.path.join(class_path, img_name)
        img = cv2.imread(img_path)

        if img is None:
            continue

        filename = os.path.splitext(img_name)[0]
        # ----- Augmentations -----
        # 1. Flip
        flipped = cv2.flip(img, 1)
        cv2.imwrite(os.path.join(output_class_path, f"{filename}_flip.png"), flipped)
        cnt += 1 
        if ( cnt > minCleanImages * numberOfAugmentationTecniques ) :
            break 
        # 2. Rotation
        angle = random.randint(-30, 30)  # rotate between -30 to +30 degrees
        rotated = rotate_image(img, angle)
        cv2.imwrite(os.path.join(output_class_path, f"{filename}_rot.png"), rotated)
        cnt += 1 
        if ( cnt > minCleanImages * numberOfAugmentationTecniques ) :
            break 
        # 3. Gaussian Noise
        noisy = add_gaussian_noise(img)
        cv2.imwrite(os.path.join(output_class_path, f"{filename}_noise.png"), noisy)
        cnt += 1 
        if ( cnt > minCleanImages * numberOfAugmentationTecniques ) :
            break 
        # 4. Brightness Change
        bright = change_brightness(img)
        cv2.imwrite(os.path.join(output_class_path, f"{filename}_bright.png"), bright)
        cnt += 1 
        if ( cnt > minCleanImages * numberOfAugmentationTecniques ) :
            break 
print("Augmentation completed successfully!")